# 26기 AI/Bigdata 아카데미 자연어처리 과목의 실습을 위한 견본 코드입니다.
본 코드는 주어진 한국어 hate-speech 데이터셋을 text classification task로 학습 및 추론을 수행합니다.

본 프로젝트에 참여하신 여러분은 코드를 수정하여 성능을 향상시키시면 됩니다.

코드 수정의 예시로는
*   하이퍼 패러미터 조정
*   전처리 과정 추가 혹은 변경 (이를테면 기사 제목 또한 학습에 사용)
*   토큰화 방법 조정

들이 있고, 이에 국한되지 않습니다.

먼저 모델과 데이터셋을 로드하고 학습하는데 필요한 패키지를 설치합니다.

In [ ]:
! pip install datasets transformers accelerate evaluate

설치후 패키지를 로드해주고, 재현성을 위해 시드를 고정해줍니다.

In [ ]:
import wandb
wandb.init(mode="disabled")

In [ ]:
import transformers
from datasets import load_dataset

transformers.set_seed(42)

## 데이터 로드

이제 데이터셋을 로드해줍니다. \
kaggle에서 본 코드를 실행하신 경우, 데이터셋이 자동으로 런타임 루트 경로의 `kaggle/input`디렉토리에 마운트됩니다.

밑의 코드로 마운트된 경로의 데이터를 읽어옵니다.

만일 기사 제목을 추가적으로 학습에 사용하고 싶으시다면, \
같은 경로의 `train.news_title.txt`, `dev.news_title.txt`, `test.news_title.txt`도 추가적으로 로드후, \
Huggingface Datasets 패키지의 [`concatenate_datasets`](https://huggingface.co/docs/datasets/process#concatenate) 메서드를 활용하여 각 스플릿에 추가해주시면 됩니다.

In [ ]:
# TODO: colab 테스트 때문에 경로 바꿈. /content/input/korean-hate-speech-detection/ 경로를 나중에 /kaggle/input/korean-hate-speech-detection/ 으로 바꾸기
train_dataset = load_dataset("csv", data_files="/kaggle/input/korean-hate-speech-detection/train.hate.csv")['train']
dev_dataset = load_dataset("csv", data_files="/kaggle/input/korean-hate-speech-detection/dev.hate.csv")['train']
test_dataset = load_dataset("csv", data_files="/kaggle/input/korean-hate-speech-detection/test.hate.no_label.csv")['train']

데이터셋의 구조를 확인해봅시다. 만일 추가한 것이 없을 경우, `train` 스플릿에서는 `comments`와 `label` column 만이 있을 것입니다.

In [ ]:
train_dataset

이제 한국어 hate-speech 데이터셋의 3가지 split `train`, `dev`, `test`을 모두 로드했습니다.

각 split에 대해 간단히 설명하자면, \
`train` split은 모델의 학습에 이용되어야 하고, \
`dev` split은 `train` split으로 학습한 모델의 hyper-parameter 조정 등을 위해 사용되어야 하고, \
`test` split은 튜닝이 완료된 최종 모델의 결과를 추론하는데 사용하는 데이터셋입니다.

`test` split의 결과는 모델 튜닝 중 사용되서는 안되며, 따라서 **레이블도 제공되지 않습니다**.

In [ ]:
train_dataset[0] # 첫번쨰 데이터 확인

## 데이터 전처리

이제 데이터셋을 전처리해줍니다.

본 코드는 예시 코드이기때문에, 간단하게 토크나이징만 진행합니다.

본 예시에서는 [`beomi/KcELECTRA-base`](https://huggingface.co/beomi/KcELECTRA-base)를 사용합니다. (모델 레포에 유용한 정보가 많이 있습니다.)\
만일 다른 모델을 사용하고 싶으시다면, [여기](https://huggingface.co/models?pipeline_tag=text-classification&language=ko)에서 다른 모델들을 해당 태스크에 적합한 다른 모델들을 찾아볼 수 있습니다.

In [ ]:
model_checkpoint = 'beomi/KcELECTRA-base'

해당 모델의 토크나이저를 로드해줍니다.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

이제 전처리에 사용할 함수를 정의해줍니다. 각 데이터는 `examples`라는 패러미터로 전달됩니다.

예시 전처리 함수는 토크나이징 및 `label` 데이터를 숫자로 매핑하는 처리만을 다룹니다.

In [ ]:
label2id = {
    'none': 0,
    'offensive': 1,
    'hate': 2,
}

In [ ]:
def preprocess_function(examples):
    result_dict = tokenizer(examples['comments'], truncation=True)
    # 추가하고 싶은 전처리 과정이 있으시다면, 여기에 추가해주시면 됩니다.
    # 만일 기사 제목 데이터를 추가하셨다면, 밑의 예시 코드를 활용하여 두 column이 합쳐진 토크나이징 결과를 얻을 수 있습니다.
    # result_dict = tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
    if 'label' in examples.keys():
      result_dict['label'] = [label2id[label_str] for label_str in examples['label']]
    return result_dict

In [ ]:
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['comments','label'])
dev_dataset = dev_dataset.map(preprocess_function, batched=True, remove_columns=['comments','label'])
test_dataset = test_dataset.map(preprocess_function, batched=True)

전처리 후, 데이터셋의 구조가 어떻게 바뀌었는지 확인해봅시다.

In [ ]:
train_dataset

이제 `train` 스플릿 기준 `input_ids`, `attention_mask`, `token_type_ids`, `label` 네 가지 column이 생긴 것을 확인하실 수 있습니다.

## 모델 학습

이제 선택한 모델을 학습해봅시다.
이를 위해 우선 모델을 로드해봅시다.

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3  # none, offensive, hate 세 가지 이므로
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

이제 학습에 사용할 하이퍼 패리미터를 정의합니다.
하이퍼 패러미터를 조정하실 분은 밑의 코드를 변경하시면 됩니다.

In [ ]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 16
learning_rate = 2e-5
epochs = 5
weight_decay = 0.01

args = TrainingArguments(
    f"{model_name}-finetuned-lr{learning_rate}-epochs{epochs}-decay{weight_decay}-hate",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    report_to="none"
)

import os
os.environ["WANDB_DISABLED"] = "true"

이후 해당 하이퍼 패러미터를 비롯한 정보들을 Huggingface Trainer에 넘겨줍니다.

In [ ]:
import evaluate

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(p):
    logits, labels = p
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

이제 학습을 진행합니다. 기본 하이퍼 패리미터에서 `NVIDIA T4` GPU 기준 약 7~8분 정도 소요됩니다.

In [ ]:
trainer.train()

학습이 완료되었다면, Trainer의 `evaluate` 메서드를 이용해 `dev` 스플릿에서의 최종 loss를 확인합니다.

In [ ]:
trainer.evaluate()

만족할 만한 결과가 나오셨나요? 해당 결과를 kaggle에 업로드 하실려면, `dev` 스플릿의 결과가 아닌 `test` 스플릿의 결과가 필요합니다.

이를 위해 `test` 스플릿에서 추론 결과를 얻고, `test_results`에 저장합니다.

In [ ]:
from tqdm import tqdm
import torch

test_results = []

for example_idx in tqdm(range(0, len(test_dataset), batch_size)):
    features = test_dataset[example_idx:example_idx + batch_size if example_idx + batch_size < len(test_dataset) else len(test_dataset)]
    batch_size = len(features['input_ids'])

    # 데이터의 배치 처리가 가능하게끔 padding을 진행합니다.
    input_ids_batch = []
    attention_mask_batch = []
    max_input_ids_len = max([len(feature) for feature in features['input_ids']])
    for input_ids, attention_mask in zip(features['input_ids'], features['attention_mask']):
        if len(input_ids) < max_input_ids_len:
            input_ids += [tokenizer.pad_token_id] * (max_input_ids_len - len(input_ids))
            attention_mask += [0] * (max_input_ids_len - len(attention_mask))
        input_ids_batch.append(input_ids)
        attention_mask_batch.append(attention_mask)
    features_batch = {'input_ids': torch.tensor(input_ids_batch).to('cuda'),
                      'attention_mask': torch.tensor(attention_mask_batch).to('cuda')}

    with torch.no_grad():
        model_output = model(**features_batch)
        preds = model_output.logits.argmax(dim=-1)
        preds = preds.squeeze().tolist()

    for comment, pred in zip(features['comments'], preds):
        test_results.append({'comments': comment, 'label': int(pred)})

모두 추론했다면, `test_result`에 저장된 결과를 csv 파일로 저장합니다.

In [ ]:
import csv

with open('./submission.csv', 'w', newline='') as f:
    fieldnames = ['comments', 'label']
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(test_results)

파일이 정확한 포맷으로 저장됬는지 확인해봅시다.


```
comments,label
(댓글 내용 1),(추론 결과 숫자)
(댓글 내용 2),(추론 결과 숫자)
...
```
의 형태로 저장되었으면 올바르게 저장된 것입니다.



In [ ]:
!head -n 10 submission.csv